In [225]:
import requests
import json
import os
from rdflib import Graph, Namespace, Literal, BNode, URIRef
from rdflib.namespace import RDF
import glob
import urllib.parse as up
from graphdbfunctions import *

In [226]:
GRAPHDB_HOST = "http://localhost:7200"
GRAPHDB_REPO = 'gentilly_cadastre' # str | The repository ID
TTL_PATH = "C:/Users/stual/PycharmProjects/peuplement_ontology/ontologie-peuplement/data/rdf"
ONTOLOGY_PATH = "D:/Documents/ONTOLOGY/"
ONTOLOGY_MODULES = ['ontology-adresse.ttl','ontology-cadastre.ttl']
NAMED_GRAPHS = ["http://data.ign.fr/sources_and_owners/",
        "http://data.ign.fr/plots/frommaps/",
        "http://data.ign.fr/plots/fromregisters/",
        "http://data.ign.fr/ontology",
        "http://data.ign.fr/otherslandmarks"]

In [227]:
MATRICE = "source:94_Gentilly_MAT_NB_1848"
LINKS_GRAPH = "<http://data.ign.fr/plots/cad2/>"

#MATRICE = "source:94_Gentilly_MAT_B_NB_1813"
#LINKS_GRAPH = "<http://data.ign.fr/plots/cad1/>"

In [228]:
PRINT = True

In [229]:
remove_named_graphs(GRAPHDB_HOST,GRAPHDB_REPO,['http://data.ign.fr/plots/cad2/'])
#remove_named_graphs(GRAPHDB_HOST,GRAPHDB_REPO,['http://data.ign.fr/plots/cad1/'])

In [230]:
prefixes = """PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX cad_ltype: <http://data.ign.fr/id/codes/cadastre/landmarkType/>
PREFIX cad: <http://data.ign.fr/def/cadastre#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX add: <http://rdf.geohistoricaldata.org/def/address#>
PREFIX cad_atype: <http://data.ign.fr/id/codes/cadastre/attributeType/>
PREFIX cad_spval: <http://data.ign.fr/id/codes/cadastrenap/specialCellValue/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>
PREFIX source: <http://data.ign.fr/id/source/>
PREFIX srctype: <http://data.ign.fr/id/codes/cadastre/sourceType/>
PREFIX cad_etype: <http://data.ign.fr/id/codes/cadastre/eventType/>"""

### 1. Create similarity links between "maps" plots and "registers" plots

In [231]:
QUERY = prefixes + """
INSERT {
        GRAPH """ + LINKS_GRAPH + """{
            ?registersLandmark add:isSimilarTo ?mapsLandmark.
            ?mapsLandmark add:isSimilarTo ?registersLandmark.}}
    WHERE {
        GRAPH <http://data.ign.fr/plots/frommaps/> {
           ?mapsLandmark a add:Landmark ; add:isLandmarkType cad_ltype:Plot.
        }
        ?registersLandmark a add:Landmark ; add:isLandmarkType cad_ltype:Plot.
        ?registersLandmark add:hasAttribute ?attr1.
        ?attr1 add:isAttributeType cad_atype:PlotMention; add:hasAttributeVersion ?attrv1.
        ?attrv1 cad:isMentionnedIn/rico:isOrWasConstituentOf+/rico:isOrWasIncludedIn  """ + MATRICE + """.
    
        MINUS {?registersLandmark add:isSimilarTo ?mapsLandmark}
        MINUS {?mapsLandmark add:isSimilarTo ?registersLandmark}
        ?mapsLandmark dcterms:identifier ?plotidm.
        ?registersLandmark dcterms:identifier ?plotidr.
        BIND(
             IF(STRENDS(STR(?plotidr), "p"), 
                SUBSTR(STR(?plotidr), 1, STRLEN(STR(?plotidr)) - 1), 
                ?plotidr
              ) AS ?plotid
            )
        filter(?plotidm = ?plotid)
}
"""
if PRINT:
  print(QUERY)

PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX cad_ltype: <http://data.ign.fr/id/codes/cadastre/landmarkType/>
PREFIX cad: <http://data.ign.fr/def/cadastre#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX add: <http://rdf.geohistoricaldata.org/def/address#>
PREFIX cad_atype: <http://data.ign.fr/id/codes/cadastre/attributeType/>
PREFIX cad_spval: <http://data.ign.fr/id/codes/cadastrenap/specialCellValue/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>
PREFIX source: <http://data.ign.fr/id/source/>
PREFIX srctype: <http://data.ign.fr/id/codes/cadastre/sourceType/>
PREFIX cad_etype: <http://data.ign.fr/id/codes/cadastre/eventType/>
INSERT {
        GRAPH <http://data.ign.fr/plots/cad2/>{
            ?registersLandmark add:isSimilarTo ?mapsLandmark.
            ?mapsLandmark add:isSimilarTo ?

In [232]:
update_sparql_query(GRAPHDB_HOST,GRAPHDB_REPO,QUERY)

<Response [204]> 


### 2. Tag Event (related to PlotMention attribute) with their types

* Les articles de classements où "Porté à" contient au moins deux numéros folios correspondent à des divisions de parcelles (cad_etype:Split)

In [233]:
QUERY = prefixes + """
INSERT {
    GRAPH """ + LINKS_GRAPH + """{
        ?eventEnd cad:isEventType cad_etype:Split.
    }
}
WHERE {
SELECT ?plot ?eventEnd
WHERE {
    ?plot a add:Landmark .
    ?plot add:isLandmarkType cad_ltype:Plot .
    ?plot dcterms:identifier ?plotid .
    ?plot add:hasAttribute ?attrMention.
    ?attrMention add:isAttributeType cad_atype:PlotMention.
    
    # Documents
    ?attrMention add:hasAttributeVersion/cad:isMentionnedIn/rico:isOrWasConstituentOf+/rico:isOrWasIncludedIn ?matrice.
    ?attrMention add:hasAttributeVersion/cad:isMentionnedIn/rico:isOrWasConstituentOf+ ?folio.
    ?attrMention add:hasAttributeVersion/cad:passedTo ?portea.
    ?portea cad:hasNumFolio ?portealabel.
    ?portea cad:isSourceType [skos:broader+ srctype:Folio].
    ?folio cad:isSourceType [skos:broader+ srctype:Folio].
    ?attrMention add:hasAttributeVersion/cad:isMentionnedIn/rico:isOrWasConstituentOf+ ?cf.
    ?cf cad:isSourceType srctype:CompteFoncier.
    
    # Changes and Events
    ?attrMention add:hasAttributeVersion/add:isMadeEffectiveBy ?changeStart.
    ?attrMention add:hasAttributeVersion/add:isOutdatedBy ?changeEnd.
    ?changeStart add:dependsOn ?eventStart.
    ?changeEnd add:dependsOn ?eventEnd.

    # Filters
    FILTER(?matrice = """ + MATRICE + """)
    }
  GROUP BY ?plot ?eventEnd
  HAVING(COUNT(distinct ?portea) > 1)
  ORDER BY ?folio ?rowid
}"""
if PRINT:
    print(QUERY)

PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX cad_ltype: <http://data.ign.fr/id/codes/cadastre/landmarkType/>
PREFIX cad: <http://data.ign.fr/def/cadastre#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX add: <http://rdf.geohistoricaldata.org/def/address#>
PREFIX cad_atype: <http://data.ign.fr/id/codes/cadastre/attributeType/>
PREFIX cad_spval: <http://data.ign.fr/id/codes/cadastrenap/specialCellValue/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>
PREFIX source: <http://data.ign.fr/id/source/>
PREFIX srctype: <http://data.ign.fr/id/codes/cadastre/sourceType/>
PREFIX cad_etype: <http://data.ign.fr/id/codes/cadastre/eventType/>
INSERT {
    GRAPH <http://data.ign.fr/plots/cad2/>{
        ?eventEnd cad:isEventType cad_etype:Split.
    }
}
WHERE {
SELECT ?plot ?eventEnd
WHERE {
    ?plot a

In [234]:
update_sparql_query(GRAPHDB_HOST,GRAPHDB_REPO,QUERY)

<Response [204]> 


* Les articles de classement où "Tiré de" qui contiennent au moins deux numéros folios correspondent à des fusions de parcelles (cad_etype:Merge)

In [235]:
QUERY = prefixes + """
INSERT {
    GRAPH """ + LINKS_GRAPH + """ {
        ?eventStart cad:isEventType cad_etype:Merge.
    }
}
WHERE {
SELECT ?plot ?eventStart
WHERE {
    ?plot a add:Landmark .
    ?plot add:isLandmarkType cad_ltype:Plot .
    ?plot dcterms:identifier ?plotid .
    ?plot add:hasAttribute ?attrMention.
    ?attrMention add:isAttributeType cad_atype:PlotMention.
    
    # Documents
    ?attrMention add:hasAttributeVersion/cad:isMentionnedIn/rico:isOrWasConstituentOf+/rico:isOrWasIncludedIn ?matrice.
    ?attrMention add:hasAttributeVersion/cad:isMentionnedIn/rico:isOrWasConstituentOf+ ?folio.
    ?attrMention add:hasAttributeVersion/cad:takenFrom ?tirede.
    ?tirede cad:hasNumFolio ?tiredelabel.
    ?tirede cad:isSourceType [skos:broader+ srctype:Folio].
    ?folio cad:isSourceType [skos:broader+ srctype:Folio].
    ?attrMention add:hasAttributeVersion/cad:isMentionnedIn/rico:isOrWasConstituentOf+ ?cf.
    ?cf cad:isSourceType srctype:CompteFoncier.
    
    # Changes and Events
    ?attrMention add:hasAttributeVersion/add:isMadeEffectiveBy ?changeStart.
    ?attrMention add:hasAttributeVersion/add:isOutdatedBy ?changeEnd.
    ?changeStart add:dependsOn ?eventStart.
    ?changeEnd add:dependsOn ?eventEnd.

    # Filters
    FILTER(?matrice = """ + MATRICE + """)
    }
  GROUP BY ?plot ?eventStart
  HAVING(COUNT(distinct ?tirede) > 1)
  ORDER BY ?folio ?rowid
}"""
if PRINT:
    print(QUERY)

PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX cad_ltype: <http://data.ign.fr/id/codes/cadastre/landmarkType/>
PREFIX cad: <http://data.ign.fr/def/cadastre#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX add: <http://rdf.geohistoricaldata.org/def/address#>
PREFIX cad_atype: <http://data.ign.fr/id/codes/cadastre/attributeType/>
PREFIX cad_spval: <http://data.ign.fr/id/codes/cadastrenap/specialCellValue/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>
PREFIX source: <http://data.ign.fr/id/source/>
PREFIX srctype: <http://data.ign.fr/id/codes/cadastre/sourceType/>
PREFIX cad_etype: <http://data.ign.fr/id/codes/cadastre/eventType/>
INSERT {
    GRAPH <http://data.ign.fr/plots/cad2/> {
        ?eventStart cad:isEventType cad_etype:Merge.
    }
}
WHERE {
SELECT ?plot ?eventStart
WHERE {
    ?p

In [236]:
update_sparql_query(GRAPHDB_HOST,GRAPHDB_REPO,QUERY)

<Response [204]> 


* Les articles de classements où "Porté à" contient exactement un numéro de folio égal au folio de l'article correspondent à une mise à jour d'un des attributs de la parcelle sans changement d'identité.

In [237]:
QUERY = prefixes + """
INSERT {
    GRAPH """ + LINKS_GRAPH + """ {
        ?eventEnd cad:isEventType cad_etype:PropertiesUpdateInSameFolio.
    }
}
WHERE {
SELECT ?plot ?eventEnd
WHERE {
    ?plot a add:Landmark .
    ?plot add:isLandmarkType cad_ltype:Plot .
    ?plot dcterms:identifier ?plotid .
    ?plot add:hasAttribute ?attrMention.
    ?attrMention add:isAttributeType cad_atype:PlotMention.
    
    # Documents
    ?attrMention add:hasAttributeVersion/cad:isMentionnedIn/rico:isOrWasConstituentOf+/rico:isOrWasIncludedIn ?matrice.
    ?attrMention add:hasAttributeVersion/cad:isMentionnedIn/rico:isOrWasConstituentOf+ ?folio.
    ?attrMention add:hasAttributeVersion/cad:passedTo ?portea.
    ?portea cad:hasNumFolio ?portealabel.
    ?portea cad:isSourceType [skos:broader+ srctype:Folio].
    ?folio cad:isSourceType [skos:broader+ srctype:Folio].
    ?attrMention add:hasAttributeVersion/cad:isMentionnedIn/rico:isOrWasConstituentOf+ ?cf.
    ?cf cad:isSourceType srctype:CompteFoncier.
    
    # Changes and Events
    ?attrMention add:hasAttributeVersion/add:isMadeEffectiveBy ?changeStart.
    ?attrMention add:hasAttributeVersion/add:isOutdatedBy ?changeEnd.
    ?changeStart add:dependsOn ?eventStart.
    ?changeEnd add:dependsOn ?eventEnd.
    OPTIONAL{?eventEnd cad:isEventType ?et.}
    FILTER(!BOUND(?et) || ?et != cad_etype:Split)

    # Filters
    FILTER(?matrice = """ + MATRICE + """)
    FILTER(?folio = ?portea)
    
    }
  GROUP BY ?plot ?eventEnd
  HAVING(COUNT(distinct ?portea) = 1)
  ORDER BY ?folio ?rowid
}"""
if PRINT:
    print(QUERY)

PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX cad_ltype: <http://data.ign.fr/id/codes/cadastre/landmarkType/>
PREFIX cad: <http://data.ign.fr/def/cadastre#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX add: <http://rdf.geohistoricaldata.org/def/address#>
PREFIX cad_atype: <http://data.ign.fr/id/codes/cadastre/attributeType/>
PREFIX cad_spval: <http://data.ign.fr/id/codes/cadastrenap/specialCellValue/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>
PREFIX source: <http://data.ign.fr/id/source/>
PREFIX srctype: <http://data.ign.fr/id/codes/cadastre/sourceType/>
PREFIX cad_etype: <http://data.ign.fr/id/codes/cadastre/eventType/>
INSERT {
    GRAPH <http://data.ign.fr/plots/cad2/> {
        ?eventEnd cad:isEventType cad_etype:PropertiesUpdateInSameFolio.
    }
}
WHERE {
SELECT ?plot ?event

In [238]:
update_sparql_query(GRAPHDB_HOST,GRAPHDB_REPO,QUERY)

<Response [204]> 


* Les articles de classements où "Porté à" contient exactement un numéro de folio **différent** du folio de l'article correspondent à un changement de compte foncier.

In [239]:
QUERY = prefixes + """
INSERT {
    GRAPH """ + LINKS_GRAPH + """ {
        ?eventEnd cad:isEventType cad_etype:FolioChange.
    }
}
WHERE {
SELECT ?plot ?eventEnd
WHERE {
    ?plot a add:Landmark .
    ?plot add:isLandmarkType cad_ltype:Plot .
    ?plot dcterms:identifier ?plotid .
    ?plot add:hasAttribute ?attrMention.
    ?attrMention add:isAttributeType cad_atype:PlotMention.
    
    # Documents
    ?attrMention add:hasAttributeVersion/cad:isMentionnedIn/rico:isOrWasConstituentOf+/rico:isOrWasIncludedIn ?matrice.
    ?attrMention add:hasAttributeVersion/cad:isMentionnedIn/rico:isOrWasConstituentOf+ ?folio.
    ?attrMention add:hasAttributeVersion/cad:passedTo ?portea.
    ?portea cad:hasNumFolio ?portealabel.
    ?portea cad:isSourceType [skos:broader+ srctype:Folio].
    ?folio cad:isSourceType [skos:broader+ srctype:Folio].
    ?attrMention add:hasAttributeVersion/cad:isMentionnedIn/rico:isOrWasConstituentOf+ ?cf.
    ?cf cad:isSourceType srctype:CompteFoncier.
    
    # Changes and Events
    ?attrMention add:hasAttributeVersion/add:isMadeEffectiveBy ?changeStart.
    ?attrMention add:hasAttributeVersion/add:isOutdatedBy ?changeEnd.
    ?changeStart add:dependsOn ?eventStart.
    ?changeEnd add:dependsOn ?eventEnd.
    OPTIONAL{?eventEnd cad:isEventType ?et.}
    FILTER(!BOUND(?et) || ?et != cad_etype:Split)

    # Filters
    FILTER(?matrice = """ + MATRICE + """)
    FILTER(?folio != ?portea)
    }
  GROUP BY ?plot ?eventEnd
  HAVING(COUNT(distinct ?portea) = 1)
  ORDER BY ?folio ?rowid
}"""
if PRINT:
    print(QUERY)

PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX cad_ltype: <http://data.ign.fr/id/codes/cadastre/landmarkType/>
PREFIX cad: <http://data.ign.fr/def/cadastre#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX add: <http://rdf.geohistoricaldata.org/def/address#>
PREFIX cad_atype: <http://data.ign.fr/id/codes/cadastre/attributeType/>
PREFIX cad_spval: <http://data.ign.fr/id/codes/cadastrenap/specialCellValue/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>
PREFIX source: <http://data.ign.fr/id/source/>
PREFIX srctype: <http://data.ign.fr/id/codes/cadastre/sourceType/>
PREFIX cad_etype: <http://data.ign.fr/id/codes/cadastre/eventType/>
INSERT {
    GRAPH <http://data.ign.fr/plots/cad2/> {
        ?eventEnd cad:isEventType cad_etype:FolioChange.
    }
}
WHERE {
SELECT ?plot ?eventEnd
WHERE {
    

In [240]:
update_sparql_query(GRAPHDB_HOST,GRAPHDB_REPO,QUERY)

<Response [204]> 


* Les articles de classements où "Tiré de" contient exactement **ResteSV** signifie que la parcelle a été divisée et qu'une partie reste attachée à ce compte foncier.

In [241]:
QUERY = prefixes + """
INSERT {
    GRAPH """ + LINKS_GRAPH + """ {
        ?eventStart cad:isEventType cad_etype:Split.
    }
}
WHERE {
SELECT ?plot ?eventStart
WHERE {
    ?plot a add:Landmark .
    ?plot add:isLandmarkType cad_ltype:Plot .
    ?plot dcterms:identifier ?plotid .
    ?plot add:hasAttribute ?attrMention.
    ?attrMention add:isAttributeType cad_atype:PlotMention.
    
    # Documents
    ?attrMention add:hasAttributeVersion/cad:isMentionnedIn/rico:isOrWasConstituentOf+/rico:isOrWasIncludedIn ?matrice.
    ?attrMention add:hasAttributeVersion/cad:isMentionnedIn/rico:isOrWasConstituentOf+ ?folio.
    ?attrMention add:hasAttributeVersion/cad:takenFrom cad_spval:Reste.
    ?folio cad:isSourceType [skos:broader+ srctype:Folio].
    ?attrMention add:hasAttributeVersion/cad:isMentionnedIn/rico:isOrWasConstituentOf+ ?cf.
    ?cf cad:isSourceType srctype:CompteFoncier.
    
    # Changes and Events
    ?attrMention add:hasAttributeVersion/add:isMadeEffectiveBy ?changeStart.
    ?attrMention add:hasAttributeVersion/add:isOutdatedBy ?changeEnd.
    ?changeStart add:dependsOn ?eventStart.
    ?changeEnd add:dependsOn ?eventEnd.

    # Filters
    FILTER(?matrice = """ + MATRICE + """)
    }
  GROUP BY ?plot ?eventStart
  HAVING(COUNT(distinct ?tirede) = 1)
  ORDER BY ?folio ?rowid
}"""
if PRINT:
    print(QUERY)

PREFIX ltype: <http://rdf.geohistoricaldata.org/id/codes/address/landmarkType/>
PREFIX skos: <http://www.w3.org/2004/02/skos/core#>
PREFIX cad_ltype: <http://data.ign.fr/id/codes/cadastre/landmarkType/>
PREFIX cad: <http://data.ign.fr/def/cadastre#>
PREFIX dcterms: <http://purl.org/dc/terms/>
PREFIX add: <http://rdf.geohistoricaldata.org/def/address#>
PREFIX cad_atype: <http://data.ign.fr/id/codes/cadastre/attributeType/>
PREFIX cad_spval: <http://data.ign.fr/id/codes/cadastrenap/specialCellValue/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX rico: <https://www.ica.org/standards/RiC/ontology#>
PREFIX source: <http://data.ign.fr/id/source/>
PREFIX srctype: <http://data.ign.fr/id/codes/cadastre/sourceType/>
PREFIX cad_etype: <http://data.ign.fr/id/codes/cadastre/eventType/>
INSERT {
    GRAPH <http://data.ign.fr/plots/cad2/> {
        ?eventStart cad:isEventType cad_etype:Split.
    }
}
WHERE {
SELECT ?plot ?eventStart
WHERE {
    ?p

In [242]:
update_sparql_query(GRAPHDB_HOST,GRAPHDB_REPO,QUERY)

<Response [204]> 
